In [9]:
from helpers import database
import pandas as pd
import re
import plotly.express as px

In [10]:
reviews = database.get_reviews()
reviews.head()

,id,recommendation_id,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,language,...,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
0,215760880,None,76561199416294706,0,6,2368,626,2368,1768152153,schinese,...,1768152264,True,0,0,0.5,0,False,False,False,False
1,215760124,None,76561199565605732,0,4,325,325,264,1768155336,english,...,1768151687,True,0,0,0.5,0,True,False,False,False
2,215758567,None,76561198057788180,0,6,1358,181,1267,1768154576,japanese,...,1768150503,False,0,0,0.5,0,True,False,False,False
3,215756568,None,76561198026330868,39,10,4201,2559,4150,1768154369,english,...,1768148999,True,0,0,0.5,0,True,False,False,False
4,215756060,None,76561199751191801,0,2,5242,5242,5242,1768148529,french,...,1768148608,True,0,0,0.5,0,True,False,False,False


In [11]:
reviews.columns

Index(['id', 'recommendation_id', 'author_steamid', 'author_num_games_owned',
       'author_num_reviews', 'author_playtime_forever',
       'author_playtime_last_two_weeks', 'author_playtime_at_review',
       'author_last_played', 'language', 'review', 'timestamp_created',
       'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny',
       'weighted_vote_score', 'comment_count', 'steam_purchase',
       'received_for_free', 'written_during_early_access',
       'primarily_steam_deck'],
      dtype='object')

In [12]:
languages = [
    "brazilian", "czech", "danish", "dutch", "english", "finnish", "french", "german",
    "hungarian", "italian", "japanese", "koreana", "latam", "norwegian", "polish",
    "portuguese", "russian", "schinese", "spanish", "swedish", "tchinese", "thai",
    "turkish", "ukrainian", "vietnamese"
]
currency = r"(?:\$|€|£|¥|₽|₩|₫|฿|₺|₹|₦|₪|₱|₴|R\$|HK\$|NT\$|C\$|A\$|ARS|BRL|CLP|MXN|COP|PEN|UYU|PYG|BOB|CRC)"

price_regex = {
    "brazilian":   rf"(?:\bpre(?:ço|co)s?\b|{currency})",
    "czech":       rf"(?:\bcen(?:a|y)\b|{currency})",
    "danish":      rf"(?:\bpris(?:er)?\b|{currency})",
    "dutch":       rf"(?:\bprijs(?:en)?\b|{currency})",
    "english":     rf"(?:\bprices?\b|{currency})",
    "finnish":     rf"(?:\b(?:hinta|hinnat)\b|{currency})",
    "french":      rf"(?:\bprix\b|{currency})",
    "german":      rf"(?:\bpreise?\b|{currency})",
    "hungarian":   rf"(?:\b(?:ár|árak|ar|arak)\b|{currency})",
    "italian":     rf"(?:\bprezz(?:o|i)\b|{currency})",
    "japanese":    rf"(?:価格|値段|{currency})",
    "koreana":     rf"(?:가격|{currency})",
    "latam":       rf"(?:\bprecios?\b|{currency})",
    "norwegian":   rf"(?:\bpris(?:er)?\b|{currency})",
    "polish":      rf"(?:\bcen(?:a|y)\b|{currency})",
    "portuguese":  rf"(?:\bpre(?:ço|co)s?\b|{currency})",
    "russian":     rf"(?:\b(?:цена|цены)\b|{currency})",
    "schinese":    rf"(?:价格|价钱|{currency})",
    "spanish":     rf"(?:\bprecios?\b|{currency})",
    "swedish":     rf"(?:\bpris(?:er)?\b|{currency})",
    "tchinese":    rf"(?:價格|價錢|{currency})",
    "thai":        rf"(?:ราคา|{currency})",
    "turkish":     rf"(?:\bfiyat(?:lar|ı)?\b|{currency})",
    "ukrainian":   rf"(?:\b(?:ціна|ціни)\b|{currency})",
    "vietnamese":  rf"(?:\b(?:giá|gia)\b|{currency})",
}

min_reviews = 100

In [13]:
rows = []
for language in languages:
    subset = reviews[reviews["language"] == language]
    regex = price_regex.get(language)
    mask = subset["review"].str.contains(regex, flags=re.IGNORECASE, na=False, regex=True)

    total = len(subset)
    mentions = int(mask.sum())
    share = mentions / total if total else 0.0

    positivity = float(subset.loc[mask, "voted_up"].mean())

    rows.append({
        "language": language,
        "total_reviews": total,
        "price_mentions": mentions,
        "price_share": share,
        "price_positivity": positivity
    })


In [14]:
agg = pd.DataFrame(rows)
agg = agg[agg["total_reviews"] >= min_reviews]
agg

,language,total_reviews,price_mentions,price_share,price_positivity
0,brazilian,997,164,0.164493,0.859756
4,english,12775,2557,0.200157,0.673054
6,french,893,178,0.199328,0.640449
7,german,748,123,0.164439,0.609756
9,italian,284,78,0.274648,0.730769
10,japanese,622,8,0.012862,0.500000
11,koreana,683,5,0.007321,0.600000
12,latam,321,39,0.121495,0.846154
14,polish,162,11,0.067901,0.636364
16,russian,404,9,0.022277,0.777778


In [16]:
price_plot = agg.sort_values("price_share", ascending=False)
price_plot

,language,total_reviews,price_mentions,price_share,price_positivity
9,italian,284,78,0.274648,0.730769
4,english,12775,2557,0.200157,0.673054
6,french,893,178,0.199328,0.640449
0,brazilian,997,164,0.164493,0.859756
7,german,748,123,0.164439,0.609756
18,spanish,1133,147,0.129744,0.761905
12,latam,321,39,0.121495,0.846154
21,thai,128,14,0.109375,0.857143
22,turkish,192,16,0.083333,0.875000
14,polish,162,11,0.067901,0.636364


In [19]:
fig = px.bar(
    price_plot,
    x="language",
    y=(price_plot["price_share"]*100).fillna(0),
    title="Price mentions by language"
)
fig.update_traces(textposition="inside")
fig.update_xaxes(title_text="Share of reviews mentioning price")
fig.update_yaxes(title_text="Language")

fig.show()

In [20]:
price_positivity_plot = agg[agg["price_mentions"] > 0].sort_values("price_positivity", ascending=False)
price_positivity_plot

,language,total_reviews,price_mentions,price_share,price_positivity
22,turkish,192,16,0.083333,0.875000
0,brazilian,997,164,0.164493,0.859756
21,thai,128,14,0.109375,0.857143
12,latam,321,39,0.121495,0.846154
16,russian,404,9,0.022277,0.777778
18,spanish,1133,147,0.129744,0.761905
9,italian,284,78,0.274648,0.730769
20,tchinese,726,13,0.017906,0.692308
4,english,12775,2557,0.200157,0.673054
6,french,893,178,0.199328,0.640449


In [26]:
fig = px.bar(
    price_positivity_plot,
    x="language",
    y=(price_positivity_plot["price_positivity"] * 100).fillna(0),
    title="Sentiment of Price-Mention Reviews by Language",
    color=(price_positivity_plot["price_positivity"] > 0.7).map({True: "Good", False: "Average/Low"}),
    color_discrete_map={"Good": "#00cc96", "Average/Low": "#ff6b6b"},
    text_auto=".1f",
    template="plotly_dark"
)

fig.update_traces(textposition="inside", textfont_size=14, textfont_color="white")
fig.update_layout(
    yaxis_title="Positivity (%)",
    yaxis_tickformat=".%",
    xaxis_tickangle=-45,
    height=650,
    showlegend=True,
    legend_title_text="Sentiment Level"
)

fig.show()

In [30]:
sentiment_price_plot = agg[agg["price_mentions"] > 0].copy()
sentiment_price_plot = sentiment_price_plot.sort_values("price_share", ascending=False).reset_index(drop=True)
sentiment_price_plot

,language,total_reviews,price_mentions,price_share,price_positivity
0,italian,284,78,0.274648,0.730769
1,english,12775,2557,0.200157,0.673054
2,french,893,178,0.199328,0.640449
3,brazilian,997,164,0.164493,0.859756
4,german,748,123,0.164439,0.609756
5,spanish,1133,147,0.129744,0.761905
6,latam,321,39,0.121495,0.846154
7,thai,128,14,0.109375,0.857143
8,turkish,192,16,0.083333,0.875000
9,polish,162,11,0.067901,0.636364


In [38]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

# 1. Bar: Sentiment (positivity)
fig.add_trace(
    go.Bar(
        x=sentiment_price_plot["language"],
        y=sentiment_price_plot["price_positivity"] * 100,
        name="Positivity (%)",
        marker=dict(
            color=sentiment_price_plot["price_positivity"] * 100,
            colorscale="RdYlGn",
            cmin=40,
            cmax=95
        ),
        text=sentiment_price_plot["price_positivity"].map(lambda x: f"{x*100:.1f}%"),
        textposition="auto",
        hovertemplate="<b>%{x}</b><br>Positivity: %{y:.1f}%<extra></extra>",
        opacity=0.85
    ),
    secondary_y=False
)

# 2. Line: Price share
fig.add_trace(
    go.Scatter(
        x=sentiment_price_plot["language"],
        y=sentiment_price_plot["price_share"] * 100,
        name="Price Mention Share (%)",
        mode="lines+markers",
        line=dict(color="#00d4ff", width=3),
        marker=dict(size=10, color="#00d4ff", symbol="circle"),
        hovertemplate="<b>%{x}</b><br>Price Share: %{y:.2f}%<extra></extra>"
    ),
    secondary_y=True
)

# Layout
fig.update_layout(
    title={
        "text": "Price Mentions & Sentiment by Language<br><sub>(Bars = Sentiment among price-mention reviews, Line = Share of reviews mentioning price)</sub>",
        "y": 0.95,
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 20}
    },
    template="plotly_dark",
    height=650,
    width=1100,
    bargap=0.22,
    hovermode="x unified",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(0,0,0,0.3)"
    ),
    margin=dict(l=60, r=80, t=100, b=100)
)

#  X axis
fig.update_xaxes(
    title_text="Language",
    tickangle=-45,
    tickfont=dict(size=12)
)

# Y axis - left
fig.update_yaxes(
    title_text="Sentiment among price-mention reviews (%)",
    ticksuffix = "%",
    range=[0, 100],
    secondary_y=False,
    gridcolor="rgba(255,255,255,0.08)"
)

# Y axis - right
max_share = max(5, (sentiment_price_plot["price_share"] * 100).max() * 1.15)
fig.update_yaxes(
    title_text="Share of reviews mentioning price (%)",
    tickformat=".1f%",
    ticksuffix = "%",
    range=[0, max_share],
    secondary_y=True,
    gridcolor="rgba(0,0,0,0)",
    showgrid=False
)

fig.show()